## Create vaccine location df with empty FIPS code column

In [315]:
# Import dependencies
import requests
import pandas as pd
import os
import json
import time
import csv

In [20]:
# files to load
vaccine_provider_locations = "vaccine_provider_locations.csv"

In [304]:
# read data file to df
locations_df = pd.read_csv(vaccine_provider_locations)
locations_df.head()

,provider_location_guid,loc_store_no,loc_phone,loc_name,loc_admin_street1,loc_admin_street2,loc_admin_city,loc_admin_state,loc_admin_zip,sunday_hours,...,in_stock,supply_level,quantity_last_updated,latitude,longitude,Category,Unnamed Column,offers_free_masks,min_age_months,min_age_years
0,26ede782-382f-4170-b941-035a31ced596,916375,909-591-7429,K's Pharmacy,12582 CENTRAL AVE,NaN,CHINO,CA,91710,NaN,...,True,0,2022-02-01,34.024523,-117.690668,covid,NaN,False,NaN,NaN
1,dbc024f6-35b5-49a8-ac6f-eef3e3a8d49b,Not applicable,(520) 612-7621,Adepa Pharmacy PLLC,8770 N THORNYDALE RD. STE 190,NaN,Tucson,AZ,85742,8:00 AM - 5:00 PM,...,True,4,2022-03-07,32.365699,-111.046180,covid,NaN,False,NaN,NaN
2,dccc041b-aa06-4193-b688-10623c3cae08,Not applicable,833-427-5634,Sojourner Recovery Services,1430 University Blvd,NaN,Hamilton,OH,45011,NaN,...,False,0,2022-04-13,39.380949,-84.561108,covid,NaN,False,NaN,NaN
3,097f3416-ac7c-420b-b099-580660c5f2c7,Not applicable,(877) 733-5824,KATY TRAIL COMM HLTH MARSHALL,1825 Atchison AVE,NaN,Marshall,MO,65340-9752,NaN,...,False,0,2021-11-08,39.099105,-93.216222,covid,NaN,False,NaN,NaN
4,358b8c64-e3c2-4175-9abf-e5e55505145a,MS1002247,(352) 897-4615,RAINBOW DISCOUNT DRUGS #MS1002247,10479 N FLORIDA AVE,NaN,CITRUS SPRINGS,FL,34434,Closed,...,False,0,2022-03-15,29.016502,-82.455008,covid,NaN,False,NaN,NaN


In [ ]:
# add empty FIPs code column
locations_df["county_FIPS_code"] = ""
locations_df.head()

## Make API call request using vaccination location coordinates to retrieve respective FIPS code

In [305]:
# base url for FCC.gov call request
base_url = "https://geo.fcc.gov/api/census/block/find?"

# test url: check to return Response [400] since it's a blank search
test = requests.get(base_url)
test

<Response [400]>

### Test call

In [313]:
# test
lat_test = 46.7216047
lng_test = -111.7519263
census = 2020

# create endpoint url
latlng_url = base_url + "latitude=" + str(lat_test) + "&longitude=" + str(lng_test) + "&censusYear=" + str(census) + "&showall=false&format=json"

# make 'Get' request for county FIPS- JSON format
FIPS_test = requests.get(latlng_url).json()
FIPS_test

{'Block': {'FIPS': '300490003003088',
  'bbox': [-111.752199, 46.720588, -111.740133, 46.727035]},
 'County': {'FIPS': '30049', 'name': 'Lewis and Clark County'},
 'State': {'FIPS': '30', 'code': 'MT', 'name': 'Montana'},
 'status': 'OK',
 'executionTime': '0'}

### Actual:

In [309]:
# fixed variables for api call request
census = 2020

# create counters and begin data logging/retrieval status  
record_count = 1
set_count = 1

print("Beginning Data Retrieval     ")
print("-----------------------------")

# iterate through df to set parameters for call
for index, row in locations_df.iterrows():
    lat = row['latitude']
    lng = row['longitude']
    
    # create endpoint url with lats and longs
    latlng_url = base_url + "latitude=" + str(lat) + "&longitude=" + str(lng) + "&censusYear=" + str(census) + "&showall=false&format=json"

    # make 'Get' request for county FIPS- JSON format
    FIPS = requests.get(latlng_url).json()

    # parse JSON to retrieve the data and add to df
    FIPS_code = FIPS['County']['FIPS']
    try:
        locations_df.loc[index, "county_FIPS_code"] = FIPS['County']['FIPS']
        
    # logging: batch calls in sets of 1000 and update counters 
        if (index % 1000 == 0 and index >= 1000):
            set_count += 1
            record_count = 1
            time.sleep(20)

        # Log record, update record_count
        print(f"Processing Record {record_count} of Set {set_count} | FIPS code provided : {FIPS_code}")
        record_count += 1
    
    # If error, skip coordinate pair
    except:
        print("Error occurred... skipping.")
        pass
    
# indicate call completion
print("\n-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")


Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | FIPS code provided : 06071
Processing Record 2 of Set 1 | FIPS code provided : 04019
Processing Record 3 of Set 1 | FIPS code provided : 39017
Processing Record 4 of Set 1 | FIPS code provided : 29195
Processing Record 5 of Set 1 | FIPS code provided : 12017
Processing Record 6 of Set 1 | FIPS code provided : 06071
Processing Record 7 of Set 1 | FIPS code provided : 34001
Processing Record 8 of Set 1 | FIPS code provided : 37131
Processing Record 9 of Set 1 | FIPS code provided : 39027
Processing Record 10 of Set 1 | FIPS code provided : 51041
Processing Record 11 of Set 1 | FIPS code provided : 22071
Processing Record 12 of Set 1 | FIPS code provided : 51520
Processing Record 13 of Set 1 | FIPS code provided : 39165
Processing Record 14 of Set 1 | FIPS code provided : 36005
Processing Record 15 of Set 1 | FIPS code provided : 48113
Processing Record 16 of Set 1 | FIPS code provided : 17031
Proce

Processing Record 141 of Set 1 | FIPS code provided : 26147
Processing Record 142 of Set 1 | FIPS code provided : 04019
Processing Record 143 of Set 1 | FIPS code provided : 47157
Processing Record 144 of Set 1 | FIPS code provided : 20015
Processing Record 145 of Set 1 | FIPS code provided : 06059
Processing Record 146 of Set 1 | FIPS code provided : 47079
Processing Record 147 of Set 1 | FIPS code provided : 25027
Processing Record 148 of Set 1 | FIPS code provided : 72109
Processing Record 149 of Set 1 | FIPS code provided : 28001
Processing Record 150 of Set 1 | FIPS code provided : 53029
Processing Record 151 of Set 1 | FIPS code provided : 40119
Processing Record 152 of Set 1 | FIPS code provided : 20173
Processing Record 153 of Set 1 | FIPS code provided : 39035
Processing Record 154 of Set 1 | FIPS code provided : 18093
Processing Record 155 of Set 1 | FIPS code provided : 22097
Processing Record 156 of Set 1 | FIPS code provided : 42063
Processing Record 157 of Set 1 | FIPS co

Processing Record 280 of Set 1 | FIPS code provided : 06071
Processing Record 281 of Set 1 | FIPS code provided : 05031
Processing Record 282 of Set 1 | FIPS code provided : 42091
Processing Record 283 of Set 1 | FIPS code provided : 29135
Processing Record 284 of Set 1 | FIPS code provided : 06071
Processing Record 285 of Set 1 | FIPS code provided : 56015
Processing Record 286 of Set 1 | FIPS code provided : 22071
Processing Record 287 of Set 1 | FIPS code provided : 34031
Processing Record 288 of Set 1 | FIPS code provided : 37097
Processing Record 289 of Set 1 | FIPS code provided : 36119
Processing Record 290 of Set 1 | FIPS code provided : 22043
Processing Record 291 of Set 1 | FIPS code provided : 36005
Processing Record 292 of Set 1 | FIPS code provided : 34015
Processing Record 293 of Set 1 | FIPS code provided : 29015
Processing Record 294 of Set 1 | FIPS code provided : 36047
Processing Record 295 of Set 1 | FIPS code provided : 22105
Processing Record 296 of Set 1 | FIPS co

Processing Record 419 of Set 1 | FIPS code provided : 06029
Processing Record 420 of Set 1 | FIPS code provided : 34013
Processing Record 421 of Set 1 | FIPS code provided : 39075
Processing Record 422 of Set 1 | FIPS code provided : 47113
Processing Record 423 of Set 1 | FIPS code provided : 18097
Processing Record 424 of Set 1 | FIPS code provided : 06089
Processing Record 425 of Set 1 | FIPS code provided : 28013
Processing Record 426 of Set 1 | FIPS code provided : 13135
Processing Record 427 of Set 1 | FIPS code provided : 34031
Processing Record 428 of Set 1 | FIPS code provided : 01117
Processing Record 429 of Set 1 | FIPS code provided : 45013
Processing Record 430 of Set 1 | FIPS code provided : 13099
Processing Record 431 of Set 1 | FIPS code provided : 01115
Processing Record 432 of Set 1 | FIPS code provided : 13245
Processing Record 433 of Set 1 | FIPS code provided : 22075
Processing Record 434 of Set 1 | FIPS code provided : 08041
Processing Record 435 of Set 1 | FIPS co

Processing Record 556 of Set 1 | FIPS code provided : 24025
Processing Record 557 of Set 1 | FIPS code provided : 06029
Processing Record 558 of Set 1 | FIPS code provided : 39041
Processing Record 559 of Set 1 | FIPS code provided : 17157
Processing Record 560 of Set 1 | FIPS code provided : 49053
Processing Record 561 of Set 1 | FIPS code provided : 56023
Processing Record 562 of Set 1 | FIPS code provided : 13027
Processing Record 563 of Set 1 | FIPS code provided : 22031
Processing Record 564 of Set 1 | FIPS code provided : 38059
Processing Record 565 of Set 1 | FIPS code provided : 12061
Processing Record 566 of Set 1 | FIPS code provided : 01103
Processing Record 567 of Set 1 | FIPS code provided : 47093
Processing Record 568 of Set 1 | FIPS code provided : 39061
Processing Record 569 of Set 1 | FIPS code provided : 29031
Processing Record 570 of Set 1 | FIPS code provided : 05045
Processing Record 571 of Set 1 | FIPS code provided : 08005
Processing Record 572 of Set 1 | FIPS co

...

Processing Record 770 of Set 3 | FIPS code provided : 06073
Processing Record 771 of Set 3 | FIPS code provided : 29077
Processing Record 772 of Set 3 | FIPS code provided : 25011
Processing Record 773 of Set 3 | FIPS code provided : 06053
Processing Record 774 of Set 3 | FIPS code provided : 20121
Processing Record 775 of Set 3 | FIPS code provided : 12053
Processing Record 776 of Set 3 | FIPS code provided : 26055
Processing Record 777 of Set 3 | FIPS code provided : 13067
Processing Record 778 of Set 3 | FIPS code provided : 01017
Processing Record 779 of Set 3 | FIPS code provided : 29023
Processing Record 780 of Set 3 | FIPS code provided : 45019
Processing Record 781 of Set 3 | FIPS code provided : 51085
Processing Record 782 of Set 3 | FIPS code provided : 20043
Processing Record 783 of Set 3 | FIPS code provided : 22017
Processing Record 784 of Set 3 | FIPS code provided : 48121
Processing Record 785 of Set 3 | FIPS code provided : 49049
Processing Record 786 of Set 3 | FIPS co

Processing Record 907 of Set 3 | FIPS code provided : 40037
Processing Record 908 of Set 3 | FIPS code provided : 47163
Processing Record 909 of Set 3 | FIPS code provided : 39023
Processing Record 910 of Set 3 | FIPS code provided : 39049
Processing Record 911 of Set 3 | FIPS code provided : 21009
Processing Record 912 of Set 3 | FIPS code provided : 13117
Processing Record 913 of Set 3 | FIPS code provided : 48027
Processing Record 914 of Set 3 | FIPS code provided : 40143
Processing Record 915 of Set 3 | FIPS code provided : 20133
Processing Record 916 of Set 3 | FIPS code provided : 21053
Processing Record 917 of Set 3 | FIPS code provided : 27047
Processing Record 918 of Set 3 | FIPS code provided : 17113
Processing Record 919 of Set 3 | FIPS code provided : 16065
Processing Record 920 of Set 3 | FIPS code provided : 48027
Processing Record 921 of Set 3 | FIPS code provided : 39113
Processing Record 922 of Set 3 | FIPS code provided : 01101
Processing Record 923 of Set 3 | FIPS co

Processing Record 48 of Set 4 | FIPS code provided : 39113
Processing Record 49 of Set 4 | FIPS code provided : 39035
Processing Record 50 of Set 4 | FIPS code provided : 26147
Processing Record 51 of Set 4 | FIPS code provided : 21037
Processing Record 52 of Set 4 | FIPS code provided : 22103
Processing Record 53 of Set 4 | FIPS code provided : 21197
Processing Record 54 of Set 4 | FIPS code provided : 53053
Processing Record 55 of Set 4 | FIPS code provided : 06095
Processing Record 56 of Set 4 | FIPS code provided : 24003
Processing Record 57 of Set 4 | FIPS code provided : 24031
Processing Record 58 of Set 4 | FIPS code provided : 20041
Processing Record 59 of Set 4 | FIPS code provided : 42027
Processing Record 60 of Set 4 | FIPS code provided : 06111
Processing Record 61 of Set 4 | FIPS code provided : 08041
Processing Record 62 of Set 4 | FIPS code provided : 16051
Processing Record 63 of Set 4 | FIPS code provided : 39145
Processing Record 64 of Set 4 | FIPS code provided : 401

Processing Record 187 of Set 4 | FIPS code provided : 06019
Processing Record 188 of Set 4 | FIPS code provided : 18133
Processing Record 189 of Set 4 | FIPS code provided : 42027
Processing Record 190 of Set 4 | FIPS code provided : 34035
Processing Record 191 of Set 4 | FIPS code provided : 26163
Processing Record 192 of Set 4 | FIPS code provided : 48157
Processing Record 193 of Set 4 | FIPS code provided : 21111
Processing Record 194 of Set 4 | FIPS code provided : 05031
Processing Record 195 of Set 4 | FIPS code provided : 26015
Processing Record 196 of Set 4 | FIPS code provided : 39035
Processing Record 197 of Set 4 | FIPS code provided : 22087
Processing Record 198 of Set 4 | FIPS code provided : 36101
Processing Record 199 of Set 4 | FIPS code provided : 39025
Processing Record 200 of Set 4 | FIPS code provided : 06065
Processing Record 201 of Set 4 | FIPS code provided : 39033
Processing Record 202 of Set 4 | FIPS code provided : 48215
Processing Record 203 of Set 4 | FIPS co

KeyError: 'County'

In [310]:
locations_df.head()

,provider_location_guid,loc_store_no,loc_phone,loc_name,loc_admin_street1,loc_admin_street2,loc_admin_city,loc_admin_state,loc_admin_zip,sunday_hours,...,supply_level,quantity_last_updated,latitude,longitude,Category,Unnamed Column,offers_free_masks,min_age_months,min_age_years,county_FIPS_code
0,26ede782-382f-4170-b941-035a31ced596,916375,909-591-7429,K's Pharmacy,12582 CENTRAL AVE,NaN,CHINO,CA,91710,NaN,...,0,2022-02-01,34.024523,-117.690668,covid,NaN,False,NaN,NaN,06071
1,dbc024f6-35b5-49a8-ac6f-eef3e3a8d49b,Not applicable,(520) 612-7621,Adepa Pharmacy PLLC,8770 N THORNYDALE RD. STE 190,NaN,Tucson,AZ,85742,8:00 AM - 5:00 PM,...,4,2022-03-07,32.365699,-111.046180,covid,NaN,False,NaN,NaN,04019
2,dccc041b-aa06-4193-b688-10623c3cae08,Not applicable,833-427-5634,Sojourner Recovery Services,1430 University Blvd,NaN,Hamilton,OH,45011,NaN,...,0,2022-04-13,39.380949,-84.561108,covid,NaN,False,NaN,NaN,39017
3,097f3416-ac7c-420b-b099-580660c5f2c7,Not applicable,(877) 733-5824,KATY TRAIL COMM HLTH MARSHALL,1825 Atchison AVE,NaN,Marshall,MO,65340-9752,NaN,...,0,2021-11-08,39.099105,-93.216222,covid,NaN,False,NaN,NaN,29195
4,358b8c64-e3c2-4175-9abf-e5e55505145a,MS1002247,(352) 897-4615,RAINBOW DISCOUNT DRUGS #MS1002247,10479 N FLORIDA AVE,NaN,CITRUS SPRINGS,FL,34434,Closed,...,0,2022-03-15,29.016502,-82.455008,covid,NaN,False,NaN,NaN,12017


In [316]:
# write df to csv file
locations_df.to_csv('vaccination_locations_with_FIPS_key.csv')